In [16]:
# !pip install pandas-datareader
# !pip install fix_yahoo_finance
import pandas as pd
import datetime
from datetime import timedelta
import pandas_datareader.data as web
import fix_yahoo_finance as yf
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
class EventStudy():
    def __init__(self,file_path):
        try:
            pd.read_csv(file_path)
        except TypeError:
            return f'Wrong File Type.'
        self.dataframe = pd.DataFrame(pd.read_csv(file_path))
        self.company = self.dataframe['Ticker']
        self.market = self.dataframe['Market']
        self.industry = self.dataframe['Industry']
        self.year = self.dataframe['Year']
        self.month = self.dataframe['Month']
        self.day = self.dataframe['Day']
        self.dict = {}
        self.reg_result = list()
        for i in range(0,len(self.market)):
            self.dict[self.company[i]] = self.market[i]
        eventdate = []
        for i in range(0,len(self.dataframe)):
            eventdate.append(datetime.datetime(self.year[i],self.month[i],self.day[i]))
        self.dataframe['EventDate'] = eventdate
        #suppose there is only one event in a month
        uniquedays = self.dataframe['EventDate'].unique()
        inter_days = []
        for i in range(0,len(uniquedays)-1):
            inter_days.append(int((uniquedays[i+1]-uniquedays[i])/np.timedelta64(1,'D')))
        self.sample_length = min(inter_days)
    
    def reg(self):
        eventdate_set = self.dataframe['EventDate'].unique()
        subdate_cumulative_abnormal_return = pd.DataFrame()
        for date in eventdate_set:
            subevent_df = self.dataframe[self.dataframe.EventDate == date]
            start_date = date - np.timedelta64(self.sample_length, 'D')
            end_date = date + np.timedelta64(self.sample_length, 'D')
            subevent_company_list = subevent_df['Ticker'].unique()
            subevent_cumulative_abnormal_return = pd.DataFrame()
            
            for company in subevent_company_list:
                market = subevent_df[subevent_df.Ticker == company]['Market'].iloc[0]
                price = web.DataReader([company, market],'yahoo', start_date,end_date)
                close_price = price['Close']
                return_rate = close_price.pct_change()
                x = return_rate[company]    
                X = sm.add_constant(x)
                y = return_rate[market]
                model = sm.OLS(y,X,missing='drop')
                result = model.fit()
#                 print(result.summary())
                (a,b) = result.params

                return_rate['Abnormal_Return'] = return_rate.apply(lambda x: x[market] - a - b * x[company], axis=1)
                subevent_cumulative_abnormal_return[f'{company}'] = return_rate['Abnormal_Return'].cumsum()[1:]
            subevent_cumulative_abnormal_return['Average_CAR'] = subevent_cumulative_abnormal_return.apply(lambda x: x.sum()/(len(subevent_company_list)),axis = 1)
            self.reg_result.append(subevent_cumulative_abnormal_return['Average_CAR'])
           
    def visualization(self):
        pass

In [17]:
instance = EventStudy('/Users/amber/Downloads/Fed events v3.csv')
instance.reg()

In [ ]:
instance.reg_result[0].plot(kind="bar")

In [21]:
COL_NUM = 2
ROW_NUM = len(instance.reg_result) + 1
import matplotlib.pyplot as plt
fig, axes = plt.subplots(ROW_NUM, COL_NUM, figsize=(120,120))

for i in range(len(instance.reg_result)):
    ax = axes[int(i/COL_NUM), i%COL_NUM]
    instance.reg_result[i].plot(kind='bar', ax=ax)

# FYI - 
plt.tight_layout()